In [1]:
import numpy as np

In [2]:
class RNNLanguageModel:
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
        
        # Vocabulaire : mapping entre mots et indices
        self.vocab_size = None          # Nombre total de mots uniques dans le corpus
        self.word_to_idx = None         # Dictionnaire: mot → index (ex: "chat" → 1)
        self.idx_to_word = None         # Dictionnaire: index → mot (ex: 1 → "chat")
        
        # Poids du RNN (initialisés dans _initialize_weights)
        self.W_x = None  # Matrice de poids: input → hidden state
        self.W_h = None  # Matrice de poids: hidden state précédent → hidden state actuel
        self.b_h = None  # Biais pour le hidden state
        self.W_y = None  # Matrice de poids: hidden state → output (logits)
        self.b_y = None  # Biais pour l'output
    
    def build_vocabulary(self,texts):
        all_words = []
        for text in texts:
            words = text.split(" ")
            for word in words:
                if word not in all_words:
                    all_words.append(word)
        
        self.word_to_idx = {}
        self.idx_to_word = {}
        
        for i, word in enumerate(all_words):
            self.word_to_idx.update({word:i})
            self.idx_to_word.update({i:word})
        
        self.vocab_size = len(all_words)
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialise les matrices de poids du RNN avec Xavier/Glorot
        
        Architecture du RNN:
        - W_x : (hidden_size, vocab_size)    - Transforme l'input one-hot en hidden state
        - W_h : (hidden_size, hidden_size)   - Propage l'information temporelle
        - b_h : (hidden_size, 1)             - Biais du hidden state
        - W_y : (vocab_size, hidden_size)    - Transforme hidden state en prédictions
        - b_y : (vocab_size, 1)              - Biais de sortie
        """
        # Initialisation Xavier/Glorot améliorée
        # Scale adapté aux dimensions d'entrée/sortie pour éviter explosion/disparition des gradients
        scale_x = np.sqrt(2.0 / (self.vocab_size + self.hidden_size))
        scale_h = np.sqrt(2.0 / (self.hidden_size + self.hidden_size))
        scale_y = np.sqrt(2.0 / (self.hidden_size + self.vocab_size))
        
        # W_x: Transforme le vecteur one-hot (vocab_size,) en hidden state (hidden_size,)
        self.W_x = np.random.randn(self.hidden_size, self.vocab_size) * scale_x
        
        # W_h: Transforme le hidden state précédent (hidden_size,) en contribution au nouveau hidden state
        self.W_h = np.random.randn(self.hidden_size, self.hidden_size) * scale_h
        
        # b_h: Biais ajouté au hidden state (permet un décalage de l'activation)
        self.b_h = np.zeros((self.hidden_size, 1))
        
        # W_y: Transforme le hidden state (hidden_size,) en logits de prédiction (vocab_size,)
        self.W_y = np.random.randn(self.vocab_size, self.hidden_size) * scale_y
        
        # b_y: Biais ajouté aux logits de sortie
        self.b_y = np.zeros((self.vocab_size, 1))
    
    def word_to_vector(self, word):
        """Convertit un mot en vecteur one-hot
        
        Args:
            word: Mot à convertir (ex: "chat")
            
        Returns:
            vector: Vecteur one-hot de taille (vocab_size, 1)
                    avec un 1 à la position du mot, 0 ailleurs
                    Ex: si "chat" a l'index 1 dans un vocab de 5 mots:
                    [[0], [1], [0], [0], [0]]
        """
        idx = self.word_to_idx[word]
        vector = np.zeros((self.vocab_size, 1))
        vector[idx] = 1
        
        return vector
    
    def forward_step(self, x_t, h_prev):
        """Calcule un pas de temps forward du RNN
        
        Formule: h_t = tanh(W_h @ h_prev + W_x @ x_t + b_h)
        
        Args:
            x_t: Input au temps t (vecteur one-hot du mot), shape (vocab_size, 1)
            h_prev: Hidden state précédent, shape (hidden_size, 1)
            
        Returns:
            h_t: Nouveau hidden state, shape (hidden_size, 1)
        """
        
        term1 = np.dot(self.W_h, h_prev)  # Contribution du passé
        term2 = np.dot(self.W_x, x_t)     # Contribution de l'input actuel
        z = term1 + term2 + self.b_h      # Combinaison linéaire
        
        h_t = np.tanh(z)  # Activation non-linéaire (tanh entre -1 et 1)
        
        return h_t
    
    def forward_sequence(self, sentence):
        """Calcule les hidden states pour tous les mots d'une phrase
        
        Args:
            sentence: Phrase d'entrée (string)
            
        Returns:
            states: Liste des hidden states, un par mot
        """
        words = sentence.split(" ")
        h = np.zeros((self.hidden_size, 1))  # Hidden state initial (vecteur de zéros)
        
        states = []
        
        for word in words:
            x_t = self.word_to_vector(word)  # One-hot encoding du mot actuel
            h = self.forward_step(x_t, h)    # Calcul du nouveau hidden state
            states.append(h)                  # Sauvegarde pour backprop
        
        return states
    
    def predict_word(self, h_t):
        """Prédit les probabilités de chaque mot à partir d'un hidden state
        
        Formule: 
            logits = W_y @ h_t + b_y
            probabilities = softmax(logits)
        
        Args:
            h_t: Hidden state actuel, shape (hidden_size, 1)
            
        Returns:
            probabilities: Vecteur de probabilités pour chaque mot, shape (vocab_size, 1)
        """
    
        logits = np.dot(self.W_y, h_t) + self.b_y  # Scores bruts pour chaque mot
        
        # Softmax avec stabilité numérique
        exp_logits = np.exp(logits - np.max(logits))  # Soustraction du max pour stabilité
        sum_exp = np.sum(exp_logits)
        probabilities = exp_logits / sum_exp  # Normalisation en probabilités (somme = 1)
    
        return probabilities
    
    def predict_next_word(self, context, top_k=10):
        """Prédit le mot suivant après un contexte donné
        
        Args:
            context: Phrase de contexte
            top_k: Nombre de mots les plus probables à afficher (défaut: 10)
        """
        states = self.forward_sequence(context)
        last_state = states[-1]  # État après le dernier mot
    
        probabilities = self.predict_word(last_state)
    
        # Trouver les top K mots les plus probables
        probs_flat = probabilities.flatten()
        top_indices = np.argsort(probs_flat)[::-1][:top_k]
        
        print(f"Après '{context}', top {top_k} prédictions :")
        for rank, idx in enumerate(top_indices, 1):
            word = self.idx_to_word[idx]
            prob = probs_flat[idx]
            print(f"  {rank}. {word:15s} -> {prob:.4f}")
    
        # Mot le plus probable
        best_idx = top_indices[0]
        best_word = self.idx_to_word[best_idx]
        best_prob = probs_flat[best_idx]
    
        print(f"\nMot le plus probable: '{best_word}' (prob: {best_prob:.4f})")
        return best_word
    
    def train_on_sentence(self, sentence, learning_rate):
        """Entraîne le RNN sur une phrase avec backpropagation through time (BPTT)
        
        Args:
            sentence: Phrase d'entraînement
            learning_rate: Taux d'apprentissage (step size pour la descente de gradient)
            
        Returns:
            total_loss: Loss totale sur la phrase
        """

        # Forward pass: calcul de tous les hidden states
        states = self.forward_sequence(sentence)
        
        # Initialisation des gradients (accumulateurs)
        dW_y = np.zeros_like(self.W_y)
        db_y = np.zeros_like(self.b_y)
        dW_x = np.zeros_like(self.W_x)
        dW_h = np.zeros_like(self.W_h)
        db_h = np.zeros_like(self.b_h)

        words = sentence.split(" ")
        total_loss = 0

        # Gradient du hidden state venant du futur (pour BPTT)
        dh_next = np.zeros((self.hidden_size, 1))

        # Backpropagation à travers le temps (de la fin vers le début)
        for i in range(len(words) - 1, 0, -1):
            h_t = states[i-1]  # État après le mot i-1 (pour prédire le mot i)
            target_word = words[i]

            probabilities = self.predict_word(h_t)

            # Calcul de la loss (cross-entropy)
            target_index = self.word_to_idx[target_word]
            target_probability = probabilities[target_index]
            loss = -np.log(target_probability + 1e-8)  # Éviter log(0)
            total_loss += loss.item()

            # Gradient de la loss par rapport à la sortie softmax
            grad_output = probabilities.copy()
            grad_output[target_index] -= 1  # Dérivée de cross-entropy + softmax

            # Gradients de W_y et b_y
            dW_y += grad_output @ h_t.T
            db_y += grad_output

            # Gradient backpropagé vers le hidden state
            # Combine le gradient de la sortie ET celui du pas de temps suivant
            grad_h_t = self.W_y.T @ grad_output + dh_next

            # Obtenir l'état précédent et l'input
            if i == 1:
                h_prev = np.zeros((self.hidden_size, 1))
            else:
                h_prev = states[i-2]
            
            x_t = self.word_to_vector(words[i-1])

            # Gradient à travers tanh: dtanh(z)/dz = 1 - tanh²(z)
            grad_tanh = 1 - h_t**2
            grad_z = grad_h_t * grad_tanh

            # Gradients de W_h, W_x et b_h
            dW_h += grad_z @ h_prev.T
            dW_x += grad_z @ x_t.T
            db_h += grad_z

            # Gradient à propager au pas de temps précédent
            dh_next = self.W_h.T @ grad_z

        # Clipping des gradients pour éviter l'explosion
        max_norm = 5.0
        for grad in [dW_y, db_y, dW_x, dW_h, db_h]:
            norm = np.linalg.norm(grad)
            if norm > max_norm:
                grad *= max_norm / norm

        # Mise à jour des poids (descente de gradient)
        self.W_y -= learning_rate * dW_y
        self.b_y -= learning_rate * db_y
        self.W_x -= learning_rate * dW_x
        self.W_h -= learning_rate * dW_h
        self.b_h -= learning_rate * db_h

        return total_loss

In [3]:
# Données d'entraînement - Corpus enrichi avec phrases plus complexes
texts = [
    # --- Thème : Vie quotidienne & Actions ---
    "le chat mange du poisson frais",
    "le chat dort sur le canapé confortable",
    "le chat joue avec la balle rouge",
    "le chien aboie très fort dans le jardin",
    "le chien court rapidement dans le parc",
    "le chien mange sa nourriture préférée",
    
    # --- Thème : Cuisine & Gastronomie ---
    "le chef cuisine une délicieuse tarte aux pommes",
    "le chef prépare un excellent plat de poisson",
    "je mange une pomme rouge et juteuse",
    "je bois un café noir très chaud",
    "je prépare le diner pour ma famille",
    "tu bois de l eau fraîche et pure",
    "elle cuisine un gâteau au chocolat fondant",
    "le gâteau est très sucré et délicieux",
    "le four est extrêmement chaud et dangereux",
    "il coupe le pain frais du matin",
    
    # --- Thème : Sport & Compétition ---
    "le joueur court très vite sur le terrain",
    "le joueur lance la balle avec précision",
    "il lance la balle très haut dans le ciel",
    "je cours rapidement dans le grand parc",
    "tu nages vite dans la piscine olympique",
    "le ballon est parfaitement rond et léger",
    "le match est finalement fini après prolongation",
    "l équipe gagne brillamment le match difficile",
    "le sport est très bon pour la santé",
    
    # --- Thème : Nature & Météo ---
    "le soleil brille intensément dans le ciel",
    "le soleil est très chaud en été",
    "la lune est magnifiquement blanche cette nuit",
    "la lune brille doucement dans la nuit",
    "le vent souffle très fort ce matin",
    "les oiseaux volent très haut dans le ciel",
    "les oiseaux chantent joliment dans les arbres",
    "la mer est profondément bleue et calme",
    "la forêt est magnifiquement verte au printemps",
    "la forêt abrite de nombreux animaux sauvages",
    
    # --- Thème : Descriptions & Observations ---
    "la table est très grande et solide",
    "la maison est confortable et spacieuse",
    "le livre est intéressant et captivant",
    "le film est passionnant et émouvant",
    "la musique est douce et relaxante",
    "le professeur explique clairement la leçon difficile",
    "l étudiant travaille sérieusement pour réussir son examen",
    
    # --- Thème : Émotions & États ---
    "je suis très content de te voir",
    "tu es vraiment fatigué après le travail",
    "il est extrêmement heureux de sa réussite",
    "elle est profondément triste de partir",
    "nous sommes fiers de nos résultats",
    
    # --- Phrases avec structures variées ---
    "quand le soleil brille je suis heureux",
    "si tu cours vite tu gagneras la course",
    "le chat et le chien jouent ensemble",
    "je mange et je bois tranquillement",
    "le chef cuisine pendant que les invités arrivent"
]

# Création et initialisation du modèle avec hidden size AUGMENTÉ
model = RNNLanguageModel(hidden_size=64)  # 64 au lieu de 20 pour plus de capacité
model.build_vocabulary(texts)

print("Vocabulaire créé avec", model.vocab_size, "mots uniques")
print(f"Architecture du modèle: hidden_size={model.hidden_size}")
print("\n=== Exemples de mots du vocabulaire ===")
sample_words = list(model.word_to_idx.keys())[:20]
print(", ".join(sample_words))

# Test du forward pass
print("\n=== Test Forward Pass ===")
sentence = "le chat mange du poisson frais"
states = model.forward_sequence(sentence)
print(f"Phrase: '{sentence}'")
print(f"Nombre d'états calculés: {len(states)}")
print(f"Dimension de chaque état caché: {states[0].shape}")

# Test de prédiction
h_t = states[0]  # État après "le"
probs = model.predict_word(h_t)
print(f"\nSomme des probabilités (doit être ~1.0): {np.sum(probs):.6f}")

Vocabulaire créé avec 182 mots uniques
Architecture du modèle: hidden_size=64

=== Exemples de mots du vocabulaire ===
le, chat, mange, du, poisson, frais, dort, sur, canapé, confortable, joue, avec, la, balle, rouge, chien, aboie, très, fort, dans

=== Test Forward Pass ===
Phrase: 'le chat mange du poisson frais'
Nombre d'états calculés: 6
Dimension de chaque état caché: (64, 1)

Somme des probabilités (doit être ~1.0): 1.000000


In [4]:
# Test du modèle non entraîné
print("=== AVANT entraînement ===")
model.predict_next_word("je bois un")

=== AVANT entraînement ===
Après 'je bois un', top 10 prédictions :
  1. chat            -> 0.0078
  2. sur             -> 0.0069
  3. en              -> 0.0068
  4. pain            -> 0.0067
  5. magnifiquement  -> 0.0067
  6. prépare         -> 0.0067
  7. et              -> 0.0066
  8. son             -> 0.0066
  9. émouvant        -> 0.0065
  10. parfaitement    -> 0.0065

Mot le plus probable: 'chat' (prob: 0.0078)


'chat'

In [5]:
# ENTRAÎNEMENT AMÉLIORÉ - Sur TOUTES les phrases du corpus
print("\n=== ENTRAÎNEMENT DU MODÈLE ===")
learning_rate = 0.05  # Learning rate augmenté (était 0.01)
epochs = 300  # Plus d'epochs pour mieux apprendre

for epoch in range(epochs):
    total_epoch_loss = 0
    
    # Entraîner sur TOUTES les phrases
    for sentence in texts:
        loss = model.train_on_sentence(sentence, learning_rate)
        total_epoch_loss += loss
    
    # Afficher la progression
    if epoch % 30 == 0:
        avg_loss = total_epoch_loss / len(texts)
        print(f"Epoch {epoch}/{epochs}: Loss moyenne = {avg_loss:.4f}")

# Dernière loss
avg_loss = total_epoch_loss / len(texts)
print(f"Epoch {epochs}/{epochs}: Loss moyenne = {avg_loss:.4f}")

print(f"\nEntraînement terminé sur {len(texts)} phrases!")
print(f"Loss finale: {avg_loss:.4f} (objectif: < 5.0 pour de bonnes prédictions)")
print("\n" + "="*70)
print("=== DÉMONSTRATION DE LA PUISSANCE DU MODÈLE ===")
print("="*70)

# Test 1: Contexte simple
print("\n[Test 1] Contexte simple - 'le chat'")
print("Attendu: 'mange', 'dort', ou 'joue' (tous présents dans le corpus)")
model.predict_next_word("le chat", top_k=5)

print("\n" + "="*70)

# Test 2: Contexte plus long avec adjectifs
print("\n[Test 2] Contexte enrichi - 'le chat mange du poisson'")
print("Attendu: 'frais' (phrase exacte dans le corpus)")
model.predict_next_word("le chat mange du poisson", top_k=5)

print("\n" + "="*70)

# Test 3: Structure sujet-verbe avec adverbes
print("\n[Test 3] Avec adverbe - 'le joueur court très'")
print("Attendu: 'vite' (de 'le joueur court très vite sur le terrain')")
model.predict_next_word("le joueur court très", top_k=5)

print("\n" + "="*70)

# Test 4: Patterns répétitifs (est + adverbe)
print("\n[Test 4] Pattern 'est très' - 'le gâteau est très'")
print("Attendu: 'sucré' ou autres adjectifs")
model.predict_next_word("le gâteau est très", top_k=5)

print("\n" + "="*70)

# Test 5: Contexte météo
print("\n[Test 5] Contexte naturel - 'le soleil brille'")
print("Attendu: 'intensément' ou 'doucement'")
model.predict_next_word("le soleil brille", top_k=5)

print("\n" + "="*70)

# Test 6: Structure complexe avec prépositions
print("\n[Test 6] Contexte avec préposition - 'les oiseaux volent très haut dans'")
print("Attendu: 'le' (de 'dans le ciel')")
model.predict_next_word("les oiseaux volent très haut dans", top_k=5)

print("\n" + "="*70)

# Test 7: Coordination
print("\n[Test 7] Coordination - 'le chat et le chien'")
print("Attendu: 'jouent' (de 'le chat et le chien jouent ensemble')")
model.predict_next_word("le chat et le chien", top_k=5)

print("\n" + "="*70)

# Test 8: Contexte émotionnel long
print("\n[Test 8] Phrase complexe - 'je suis très content de'")
print("Attendu: 'te' (de 'je suis très content de te voir')")
model.predict_next_word("je suis très content de", top_k=5)

print("\n" + "="*70)
print("\n✓ Tests terminés!")
print(f"Le modèle (hidden_size={model.hidden_size}) a appris sur {epochs} epochs")
print(f"avec un learning rate de {learning_rate}")


=== ENTRAÎNEMENT DU MODÈLE ===
Epoch 0/300: Loss moyenne = 30.5521
Epoch 30/300: Loss moyenne = 5.1141
Epoch 60/300: Loss moyenne = 3.1900
Epoch 90/300: Loss moyenne = 2.8003
Epoch 120/300: Loss moyenne = 2.7187
Epoch 150/300: Loss moyenne = 2.6714
Epoch 180/300: Loss moyenne = 2.6319
Epoch 210/300: Loss moyenne = 2.6051
Epoch 240/300: Loss moyenne = 2.5840
Epoch 270/300: Loss moyenne = 2.5657
Epoch 300/300: Loss moyenne = 2.5500

Entraînement terminé sur 52 phrases!
Loss finale: 2.5500 (objectif: < 5.0 pour de bonnes prédictions)

=== DÉMONSTRATION DE LA PUISSANCE DU MODÈLE ===

[Test 1] Contexte simple - 'le chat'
Attendu: 'mange', 'dort', ou 'joue' (tous présents dans le corpus)
Après 'le chat', top 5 prédictions :
  1. et              -> 0.3940
  2. joue            -> 0.2422
  3. dort            -> 0.1956
  4. mange           -> 0.1524
  5. chef            -> 0.0026

Mot le plus probable: 'et' (prob: 0.3940)


[Test 2] Contexte enrichi - 'le chat mange du poisson'
Attendu: 'frais'